In [1]:
#------------------------------------------------------------------------
# WES 269 SBUS Demo Code
#
# This code serves as a baseline for configuring the UART16550 core
# and polling data from the RX FIFO
#-------------------------------------------------------------------------

from pynq import Overlay
from pynq import MMIO
import time
import numpy as np

In [72]:
# Load bitstream
ol = Overlay('SBUS_DEMO_ext_rcvr_clk.bit')

# Download bitstream to FPGA
t_before_bitstream = time.time()
ol.download()
t_after_bitstream = time.time()
print('  ')
print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')

  
0.3280010223388672 seconds to program bitstream


In [73]:
# Address space definitions

IP_BASE_ADDRESS = 0X43C00000
ADDRESS_RANGE = 0x10000

RX_BUF_REG       =   0x1000        #Receiver Buffer Register              LCR[7] = 0
TX_HOLD_REG      =   0x1000        #Transmitter Holding Register          LCR[7] = 0
INT_EN_REG       =   0x1004        #Interrupt Enable Register             LCR[7] = 0
INT_ID_REG       =   0x1008        #Interrupt Identification Register     LCR[7] = x
FIFO_CNTRL_REG   =   0x1008        #FIFO Control Register                 LCR[7] = x (write), LCR[7] = 1 (read)
LINE_CNTRL_REG   =   0x100C        #Line Control Register                 LCR[7] = x
MODEM_CNTRL_REG  =   0x1010        #Modem Control Register                LCR[7] = x
LINE_STATUS_REG  =   0x1014        #Line Status Register                  LCR[7] = x
MODEM_STATUS_REG =   0x1018        #Modem Status Register                 LCR[7] = x
SCRATCH_REG      =   0x101C        #Scratch Register                      LCR[7] = x
DIV_LATCH_LSB    =   0x1000        #Divisor Latch LSB Register            LCR[7] = 1
DIV_LATCH_MSB    =   0x1004        #Divisor Latch MSB Register            LCR[7] = 1

NUM_BYTES        =   25
START_BYTE     =   0xF0
STOP_BYTE      =   0x00

print('Address Definitions complete')

Address Definitions complete


In [74]:
# configuring UART16550 core

UART_core = MMIO(IP_BASE_ADDRESS,ADDRESS_RANGE)

# enable divisor latch, set baud divisor
UART_core.write(LINE_CNTRL_REG, 0x00000080)
#UART_core.write(DIV_LATCH_LSB, 0x3E) # divide by 62 for 100MHz AXI clock
UART_core.write(DIV_LATCH_LSB, 0x00000001) # divide by 1 for external 1.6 MHz BAUD rate clock
#UART_core.write(DIV_LATCH_LSB, 0x00000002) # divide by 2 for external 3.2 MHz BAUD rate clock
UART_core.write(DIV_LATCH_MSB, 0x00000000)

# configure data exchange, 100k BAUD, even parity, 2 stop bits, 8 data bits
UART_core.write(LINE_CNTRL_REG, 0x00000000)
UART_core.write(LINE_CNTRL_REG, 0x0000001F)

# enable and configure FIFO
#UART_core.write(LINE_CNTRL_REG, 0x40)
#UART_core.write(FIFO_CNTRL_REG, 0x05)

# reset FIFOs
#UART_core.write(FIFO_CNTRL_REG, 0x07)
#UART_core.write(FIFO_CNTRL_REG, 0x05)
#UART_core.write(LINE_CNTRL_REG, 0x40)

# setting WHO_AM_I value
UART_core.write(SCRATCH_REG,0x45)


print('UART16550 Core Configuration Complete')

UART16550 Core Configuration Complete


In [75]:
# reset FIFOs

#UART_core.write(FIFO_CNTRL_REG, 0x7)
#UART_core.write(FIFO_CNTRL_REG, 0x5)

print('Reset UART FIFOs')


Reset UART FIFOs


In [76]:
# allocating storage array for data

ARRAY_SIZE = 50000

buf_data = np.empty([ARRAY_SIZE,], dtype=np.int8)

print('Array allocation complete')


Array allocation complete


In [115]:
# grabbing data from the UART bus

NUM_BYTES = 2

BIT_ONE = 0x00000001
buffer = [0]*(NUM_BYTES)
BYTES_READ = 0


# resetting RX FIFO
#UART_core.write(FIFO_CNTRL_REG, 0x7)

while(BYTES_READ != (NUM_BYTES)):
    
    #read from the RX FIFO when data present
    while((UART_core.read(LINE_STATUS_REG) & BIT_ONE) != 1):
        pass
    
    buffer[0] = UART_core.read(RX_BUF_REG)
    #if ( buffer[0] == START_BYTE):
    if ( buffer[0]):
        BYTES_READ += 1
        for i in range(NUM_BYTES - 1):
            buffer[BYTES_READ] = UART_core.read(RX_BUF_REG)
            BYTES_READ += 1
    else:
        pass
    #print('Total bytes read: ', BYTES_READ)

    

    
print(buffer)
print('')
print(''.join('%02x'%i for i in buffer))   # printing buffer contents


[62, 62]

3e3e


In [93]:
# testing

WHO_AM_I = UART_core.read(SCRATCH_REG)

print('WHO_AM_I: ' + str(WHO_AM_I))

WHO_AM_I: 69


In [51]:
# Write OFDM Receiver results to output file for easier comparison
with open('output_compare.dat', 'w') as file_out:
    file_out.write("-----------------------------------------------------------------\n")
    file_out.write("             Golden Output versus OFDM Receiver Output           \n")
    file_out.write("-----------------------------------------------------------------\n\n")
    for k in range(FSIZE):
        file_out.write("index "+str(k)+",\t R: "+str(inp_R[k])+",\t I: "+str(inp_I[k])+",\t Gold: "+str(golden_D[k])+",\t OFDM: "+str(out_D[k])+"\n")

NameError: name 'FSIZE' is not defined

In [106]:
# parsing buffered packet into channel data

# channel 1   Throttle
# channel 2   Roll
# channel 3   Pitch
# channel 4   yaw

NUM_CHANNELS = 4

channel = [0]*NUM_CHANNELS

# bit masking to parse 11 bit channel data from buffer
channel[0] = (buffer[1]      | buffer[2] << 8)                    & 0x7FF
channel[1] = (buffer[2] >> 3 | buffer[3] << 5)                    & 0x7FF
channel[2] = (buffer[3] >> 6 | buffer[4] << 2 | buffer[5] << 10)  & 0x7FF
channel[3] = (buffer[5] >> 1 | buffer[6] << 7)                    & 0x7FF


print('Throttle Command:  ', channel[0])
print('Roll Command:      ', channel[1])
print('Pitch Command:     ', channel[2])
print('Yaw Command:       ', channel[3])



Throttle Command:   1614
Roll Command:       1097
Pitch Command:      1939
Yaw Command:        31


In [ ]:
#########################################
# old code
#########################################

# grabbing data from the UART bus

BIT_ONE = 0x00000001
buffer = [0]*(NUM_BYTES)
#buffer = [0]*(NUM_BYTES-1)
BYTES_READ = 0

test = [0]*(NUM_BYTES)


while(BYTES_READ != (NUM_BYTES)):
#while(BYTES_READ != (NUM_BYTES - 1)):
    
    #read from the RX FIFO when data present
    #while((mmio.read(LINE_STATUS_REG) & BIT_ONE) != 1):
        #pass
    
    #print(mmio.read(RX_BUF_REG))
    #print(idx)
    
    #if(mmio.read(RX_BUF_REG) != 240):
        #pass
        #print('hard pass')
        
        
    for index in range(NUM_BYTES):
        buffer[index] = mmio.read(RX_BUF_REG)
        BYTES_READ += 1
        
    #temp = mmio.read(RX_BUF_REG)
    #if (temp != 240):
    #    print(temp)
    #else:
    #    buffer[BYTES_READ] = mmio.read(RX_BUF_REG)
    #    BYTES_READ += 1

print(buffer) 
print('')
print(''.join('%02x'%i for i in buffer))
print('')

for i in range(NUM_BYTES):
    test[i] = ~buffer[i]
    
print(test)
print('')
print(''.join('%02x'%i for i in test))

In [ ]:
# testing grabbing channel 1

print('buffer[1]:             ', buffer[1])
print('buffer[2]:             ', buffer[2])
print('buffer[2] last 3 bits: ', buffer[2] & 0x07)

temp = buffer[1] | (buffer[2] << 8)
temp2 = temp & 0x07FF
temp3 = (buffer[1] | buffer[2] << 8) & 0x07FF
 
print('temp:                  ', temp)
print('temp2:                 ', temp2)
print('temp3:                 ', temp3)

start = buffer[0]
print('start byte:            ', start)
channel_1 = (buffer[1] | ( buffer[2] << 8 )) & 0x07FF

print('channel 1:             ', channel_1)

In [ ]:
# TRAVIS IDEAS for infinite graph display

while !quit:
  get packet
  update the graph
  send stuff if the user wants to send stuff
    i.e. break commands